In [ ]:
# we load the things!

from ngsolve import *
from ngsolve.meshes import MakeStructured3DMesh
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.geom2d import SplineGeometry
from ngsolve.solvers import GMRes

In [ ]:
def createGeometryStructuredBrick(n):
    structuredMeshUnitBrick = MakeStructured3DMesh(hexes=False, nx=n, ny=n, nz=n)
    return structuredMeshUnitBrick

def createReentrantCornerGeometry(hmax):

    largeBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0.5, 0.5, 0.5))
    smallBrick = Box(Pnt(-0.5, -0.5,-0.5), Pnt(0, 0, 0))

    reentrantCornerGeo3D = largeBrick - smallBrick
    
    reentrantCornerGeo3D.faces.Min(X).name = "minX"
    reentrantCornerGeo3D.faces.Max(X).name = "maxX"
    reentrantCornerGeo3D.faces.Min(Y).name = "minY"
    reentrantCornerGeo3D.faces.Max(Y).name = "maxY"
    reentrantCornerGeo3D.faces.Min(Z).name = "minZ"
    reentrantCornerGeo3D.faces.Max(Z).name = "maxZ"

    mesh = Mesh(OCCGeometry(reentrantCornerGeo3D).GenerateMesh(maxh=hmax))

    return mesh

In [ ]:
useGMRes = True

def hodgeLaplace2Forms(mesh,
                       f = CF((0,0,0)), # this is the right hand side f
                       order = 1,
                       C_w = 1,
                       dirichletBnd = None,
                       gOnDirichletBnd = None,
                       ):
    
    allBNDstrings = list(mesh.GetBoundaries())
    nonDirichletBNDstrings = '|'.join(sorted(set(allBNDstrings) - set(dirichletBnd)))

    H_curl = HCurl(mesh, order=order, type1=True)  # For 1-forms, H(curl) space
    H_div = HDiv(mesh, order=order-1, RT=True)     # For 2-forms, H div space
    fes = H_curl * H_div
    (p, u), (q, v) = fes.TnT()

    n = specialcf.normal(mesh.dim)
    h = specialcf.mesh_size
    dS = ds(skeleton=True)
                      
        
    B, F  = BilinearForm(fes), LinearForm(fes)

    B += curl(p) * v * dx
    B += div(u) * div(v) * dx
    B += curl(q) * u * dx
    B += - p * q * dx

    F += f * v * dx
    
    dSnonDirichlet = ds(skeleton=True, definedon=mesh.Boundaries(nonDirichletBNDstrings))
    B += Cross(n, q) * u * dSnonDirichlet

    for boundary_name, boundary_g in zip(dirichletBnd, gOnDirichletBnd):
        dS_dirichlet = ds(skeleton=True, definedon=mesh.Boundaries(boundary_name))
        B += (C_w/h) * (v*n) * (u*n) * dS_dirichlet
        B += - div(u) * (v*n) * dS_dirichlet
        B += - div(v) * (u*n) * dS_dirichlet

        
        F += - div(v) * (boundary_g*n) * dS_dirichlet
        F += (C_w/h) * (boundary_g*n) * (v*n) * dS_dirichlet
        F += Cross(n, q) * boundary_g * dS_dirichlet
    
    with TaskManager(): 
        if (useGMRes == False):
            B.Assemble()
            F.Assemble()
            sol = GridFunction(fes)
            res = F.vec-B.mat * sol.vec
            inv = B.mat.Inverse(freedofs=fes.FreeDofs(), inverse="pardiso")
            sol.vec.data += inv * res
        else:
            B.Assemble()
            F.Assemble()
            sol = GridFunction(fes)
            blocks = fes.CreateSmoothingBlocks()
            prebj = B.mat.CreateBlockSmoother(blocks)
            GMRes(A =B.mat,x= sol.vec, b=F.vec,pre = prebj,  printrates="\r", maxsteps = 10000, tol=1e-8)
            res = CF((0,0,0))

    gf_p , gf_u = sol.components
    return gf_u, gf_p

In [ ]:

mesh = createReentrantCornerGeometry(0.1)
C_w = 1000
order = 2

n = specialcf.normal(mesh.dim)
t = specialcf.tangential(mesh.dim)

zero = CF((0,0,0))

print(mesh.GetBoundaries())

bndList = ['minX', 'minY', 'minZ',  'maxY', 'maxX', 'maxZ', 'default']
gList = [-0.1*n, 0.1*n, 0.1*n, zero, zero, zero, zero]

f = 1 * CF((3, -3, -3))

gfu, gfp = hodgeLaplace2Forms(mesh, f, order, C_w, bndList, gList)
clipping ={"pnt":(0,0,-0.01), "function":True, "vec":(0,0,-1)}
Draw(gfu, mesh, clipping=clipping)
